In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import loadtxt

import sklearn
import tensorflow as tf
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Dropout

In [2]:
model_1 = pd.read_csv(r'C:/Data/Sentiment Analysis/MELD/ensemble preds/2d_cnn/v3/test_mfcc_preds.csv')
model_2 = pd.read_csv(r'C:/Data/Sentiment Analysis/MELD/ensemble preds/2d_cnn/v3/test_specgram_preds.csv')
model_3 = pd.read_csv(r'C:/Data/Sentiment Analysis/MELD/ensemble preds/wav2vec/v2/test_preds_w2vb.csv')
model_4 = pd.read_csv(r'C:/Data/Sentiment Analysis/MELD/ensemble preds/wav2vec/v2/test_preds_w2vl.csv')
model_5 = pd.read_csv(r'C:/Data/Sentiment Analysis/MELD/ensemble preds/roberta_text/v2/test_preds_roberta_v2.csv')
model_6 = pd.read_csv(r'C:/Data/Sentiment Analysis/MELD/ensemble preds/roberta_text/v2/test_preds_roberta_lfl_v2.csv')

In [3]:
#dropping first unnamed columns
model_1 = model_1.drop(columns=model_1.columns[0])
model_2 = model_2.drop(columns=model_2.columns[0])
model_3 = model_3.drop(columns=model_3.columns[0])
model_4 = model_4.drop(columns=model_4.columns[0])
model_5 = model_5.drop(columns=model_5.columns[0])
model_6 = model_6.drop(columns=model_6.columns[0])

In [4]:
model_2

,fileID_specgram,Negative_specgram,Neutral_specgram,Positive_specgram,predicted_specgram,actual_specgram
0,dia0_utt0.wav,0.264094,0.591943,0.143963,1,2
1,dia0_utt1.wav,0.407919,0.395113,0.196967,0,0
2,dia0_utt2.wav,0.271768,0.589855,0.138377,1,1
3,dia1_utt0.wav,0.340915,0.442386,0.216699,1,1
4,dia1_utt1.wav,0.315723,0.581280,0.102997,1,2
...,...,...,...,...,...,...
2603,dia279_utt11.wav,0.537996,0.298764,0.163240,0,1
2604,dia279_utt12.wav,0.179114,0.646331,0.174555,1,0
2605,dia279_utt13.wav,0.289139,0.617335,0.093526,1,1
2606,dia279_utt14.wav,0.271975,0.616445,0.111580,1,1


In [5]:
#fileID
fileID = model_1.fileID_mfcc

#target variable
target = model_1.actual_mfcc

#### Option 1:

In [6]:
#Using preds as (0, 1, 2) only 
#{'negative': 0, 'neutral': 1, 'positive': 2}
mfcc = model_1.drop(model_1.columns[[0, 1, 2, 3, 5]], axis=1)
specgram = model_2.drop(model_2.columns[[0, 1, 2, 3, 5]], axis=1)
w2vb = model_3.drop(model_3.columns[[0, 1, 2, 3, 5]], axis=1)
w2vl = model_4.drop(model_4.columns[[0, 1, 2, 3, 5]], axis=1)
roberta = model_5.drop(model_5.columns[[0, 1, 2, 3, 5]], axis=1)
roberta_lfl = model_6.drop(model_6.columns[[0, 1, 2, 3, 5]], axis=1)

In [7]:
print(mfcc)

      predicted_mfcc
0                  1
1                  1
2                  1
3                  1
4                  0
...              ...
2603               0
2604               1
2605               1
2606               1
2607               0

[2608 rows x 1 columns]


#### Option 2: 

In [8]:
#Using preds as an array
MFCC = model_1.drop(model_1.columns[[0,4,5]], axis=1)
SPECGRAM = model_2.drop(model_2.columns[[0,4,5]], axis=1)
W2VB = model_3.drop(model_3.columns[[0,4,5]], axis=1)
W2VL = model_4.drop(model_4.columns[[0,4,5]], axis=1)
ROBERTA = model_5.drop(model_5.columns[[0,4,5]], axis=1)
ROBERTA_LFL = model_6.drop(model_6.columns[[0,4,5]], axis=1)

In [9]:
W2VB

,Negative_w2vb,Neutral_w2vb,Positive_w2vb
0,0.560904,0.437992,0.001103
1,0.118941,0.880212,0.000847
2,0.448627,0.391304,0.160070
3,0.298236,0.608287,0.093476
4,0.376820,0.323143,0.300037
...,...,...,...
2603,0.736317,0.261845,0.001838
2604,0.636589,0.362769,0.000642
2605,0.775086,0.219628,0.005286
2606,0.170367,0.799375,0.030259


In [10]:
for cls in (mfcc,specgram,w2vb,w2vl,roberta,roberta_lfl):
    print(cls.columns[0], ":", accuracy_score(cls, target))

predicted_mfcc : 0.5034509202453987
predicted_specgram : 0.5130368098159509
predicted_w2vb : 0.49348159509202455
predicted_w2vl : 0.495398773006135
predicted_roberta : 0.7105061349693251
predicted_roberta_lfl : 0.7059049079754601


## Neural Network

In [11]:
#input
MFCC = model_1.drop(model_1.columns[[0,4,5]], axis=1)
SPECGRAM = model_2.drop(model_2.columns[[0,4,5]], axis=1)
W2VB = model_3.drop(model_3.columns[[0,4,5]], axis=1)
W2VL = model_4.drop(model_4.columns[[0,4,5]], axis=1)
ROBERTA = model_5.drop(model_5.columns[[0,4,5]], axis=1)
ROBERTA_LFL = model_6.drop(model_6.columns[[0,4,5]], axis=1)

dfs = [MFCC, SPECGRAM, W2VB, W2VL, ROBERTA, ROBERTA_LFL]
inp = pd.concat(dfs,axis=1)
inputs = np.array(inp)

#output/target
ar = model_1.actual_mfcc
outputs = np.array(ar)
outputs = tf.keras.utils.to_categorical(outputs) #Converting labels to one-hot encoded vectors
#outputs = arr.reshape(len(arr), 1)

In [12]:
inp

,Negative_mfcc,Neutral_mfcc,Positive_mfcc,Negative_specgram,Neutral_specgram,Positive_specgram,Negative_w2vb,Neutral_w2vb,Positive_w2vb,Negative_w2vl,Neutral_w2vl,Positive_w2vl,Negative_roberta,Neutral_roberta,Positive_roberta,Negative_roberta_lfl,Neutral_roberta_lfl,Positive_roberta_lfl
0,0.217635,0.604405,0.177960,0.264094,0.591943,0.143963,0.560904,0.437992,0.001103,0.310302,0.688611,0.001087,0.999682,0.000239,0.000079,9.858891e-01,0.007293,0.006818
1,0.340150,0.419823,0.240027,0.407919,0.395113,0.196967,0.118941,0.880212,0.000847,0.189410,0.809118,0.001471,0.931104,0.049291,0.019605,6.469994e-01,0.345483,0.007518
2,0.214909,0.624270,0.160820,0.271768,0.589855,0.138377,0.448627,0.391304,0.160070,0.082342,0.905347,0.012311,0.000063,0.999898,0.000039,1.189144e-06,0.999975,0.000024
3,0.334773,0.439070,0.226156,0.340915,0.442386,0.216699,0.298236,0.608287,0.093476,0.517103,0.341875,0.141022,0.000088,0.999744,0.000168,6.497069e-06,0.999917,0.000076
4,0.507886,0.200311,0.291804,0.315723,0.581280,0.102997,0.376820,0.323143,0.300037,0.446322,0.229367,0.324312,0.025766,0.000579,0.973656,6.001037e-03,0.000080,0.993919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2603,0.841560,0.098446,0.059994,0.537996,0.298764,0.163240,0.736317,0.261845,0.001838,0.472847,0.516096,0.011057,0.774982,0.152441,0.072576,6.796182e-02,0.786170,0.145868
2604,0.106355,0.763529,0.130116,0.179114,0.646331,0.174555,0.636589,0.362769,0.000642,0.801059,0.155648,0.043293,0.999689,0.000200,0.000111,9.998496e-01,0.000132,0.000019
2605,0.316587,0.487929,0.195484,0.289139,0.617335,0.093526,0.775086,0.219628,0.005286,0.786060,0.168378,0.045562,0.092715,0.035348,0.871938,5.459638e-01,0.333394,0.120643
2606,0.143595,0.603400,0.253005,0.271975,0.616445,0.111580,0.170367,0.799375,0.030259,0.392049,0.553376,0.054575,0.998271,0.000276,0.001453,9.309784e-01,0.001439,0.067583


In [13]:
inputs

array([[2.17634931e-01, 6.04405344e-01, 1.77959740e-01, ...,
        9.85889077e-01, 7.29268137e-03, 6.81818323e-03],
       [3.40150148e-01, 4.19823021e-01, 2.40026876e-01, ...,
        6.46999419e-01, 3.45483065e-01, 7.51755200e-03],
       [2.14909330e-01, 6.24270380e-01, 1.60820290e-01, ...,
        1.18914352e-06, 9.99974728e-01, 2.40635600e-05],
       ...,
       [3.16587180e-01, 4.87928867e-01, 1.95483923e-01, ...,
        5.45963824e-01, 3.33393544e-01, 1.20642602e-01],
       [1.43595159e-01, 6.03399813e-01, 2.53005028e-01, ...,
        9.30978417e-01, 1.43896975e-03, 6.75825924e-02],
       [5.46680629e-01, 3.74486625e-01, 7.88327456e-02, ...,
        3.55614674e-07, 9.99987245e-01, 1.24084381e-05]])

In [14]:
inputs.shape

(2608, 18)

In [15]:
outputs

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [16]:
outputs.shape

(2608, 3)

In [17]:
X = inputs
y = outputs
#define the keras model
model = Sequential()
model.add(Dense(512, input_dim=18, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

#compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#fit the keras model on the dataset
model.fit(X, y, epochs=250, batch_size=10)

#evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/250
261/261 [==============================] - 5s 7ms/step - loss: 1.1595 - accuracy: 0.5978
Epoch 2/250
261/261 [==============================] - 2s 8ms/step - loss: 0.9500 - accuracy: 0.6465
Epoch 3/250
261/261 [==============================] - 2s 8ms/step - loss: 0.8589 - accuracy: 0.6733
Epoch 4/250
261/261 [==============================] - 2s 8ms/step - loss: 0.8367 - accuracy: 0.6883
Epoch 5/250
261/261 [==============================] - 2s 8ms/step - loss: 0.8107 - accuracy: 0.6840
Epoch 6/250
261/261 [==============================] - 2s 8ms/step - loss: 0.7955 - accuracy: 0.7044
Epoch 7/250
261/261 [==============================] - 2s 8ms/step - loss: 0.7862 - accuracy: 0.7021
Epoch 8/250
261/261 [==============================] - 2s 8ms/step - loss: 0.7748 - accuracy: 0.7063
Epoch 9/250
261/261 [==============================] - 2s 8ms/step - loss: 0.7885 - accuracy: 0.7013
Epoch 10/250
261/261 [==============================] - 2s 9ms/step - loss: 0.7808 - accura